In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [7]:
auto_enc= tf.keras.models.load_model("batch_128_v4") #load the autoenc
enc= tf.keras.models.Sequential([lay for lay in auto_enc.layers[0:10]]) #make the encoder

for idx, lay in enumerate(enc.layers):  #assign trained weights to enc
    enc.layers[idx].set_weights(auto_enc.layers[idx+1].get_weights())   
#enc.summary()

In [11]:
def vid2array(vid_objs, start_frame, end_frame): #Takes in a list of cv2 video capture obj and returns a normalized np array of dim: (None, 64, 128, 128, 1)
    frame_tot= 64
    frame_width= 128
    frame_height= 128
    frame_channel= 1 #we consider grayscale image
    X= np.zeros(shape=(len(vid_objs), frame_tot,frame_width,frame_height,frame_channel)) #shape of training X

    for vid_no, video_item in enumerate(vid_objs):
        count=0
        frames_processed= 0
        while (vid_objs[vid_no].isOpened()):
            ret, frame= vid_objs[vid_no].read()
            if (ret==True):
                if (count>=start_frame and count<end_frame): 
                    resized_frame = cv2.resize(frame, (frame_height,frame_width))
                    grayFrame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY).reshape(((frame_height,frame_width, 1)))
                    X[vid_no, frames_processed, :, :, :]= grayFrame/255.0
                    frames_processed=frames_processed+1
                count=count+1
            else:
                break
        vid_objs[vid_no].release() #release the capt object
    print("Tot Frames Processed: " + str(frames_processed))
    print("Counter terminated at: " + str(count))

    #release All Vid objects 
    for vid in vid_objs:
        vid.release()

    return X

[ WARN:0@189.711] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


In [15]:
cap= cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame= cap.read()
    if ret==True:
        cv2.imshow("Live Feed", frame)
        if cv2.waitKey(1) == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@834.516] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index
